In [1]:
import os
from itertools import product
from pandas import concat, DataFrame, set_option
from seaborn import set_theme
os.chdir("../")
from src.utils import _plot_confusion_matrix
from src.data import Format
from src.models import Pipeline

2023-04-02 11:13:13.036295: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 11:13:14.046874: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-02 11:13:14.046959: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-02 11:13:14.046968: W 

In [2]:
set_option('display.max_columns', None)
set_theme()

# Prediction Global Report

In [3]:
""" initialize variables """
_list_datasets_T = [["mrda", 50],
                    ["swda", 50],
                    ["dyda_da", 5],
                    ["dyda_e", 5],
                    ["meld_s", 5],
                    ["meld_e", 5]]

_list_encoder = ["bert-base-cased", "xlnet-base-cased"]

_list_data_format_decoder = [["MLP", "concatenate"], ["GRU", "separate"]]

_list_inputs = list(product(_list_datasets_T, _list_encoder, _list_data_format_decoder))

_possible_pipelines = {"inputs": _list_inputs, "outputs": []}

df_global_report = DataFrame()

In [4]:
for _input in _possible_pipelines["inputs"]:
    """compile the model"""
    _pipeline = Pipeline(_input[0][0], _input[0][1], _input[1], _input[2][0], _input[2][1])
    _pipeline.summarize()

    """store outputs"""
    _pipeline.df_report.to_csv(
                 f"./data/outputs_data/df_{_input[0][0]}_{_input[1].split('-')[0]}_{_input[2][0]}.csv",
                 index=False,
                 sep="|",
                 encoding='utf-8')
    df_global_report = concat([df_global_report, _pipeline.df_report]).reset_index(drop=True)
    _possible_pipelines["outputs"].append(_pipeline.confusion_matrix)

    """plot and save the confusion matrix"""
    _list_labels = Format(_input[0][0], _input[0][1], _input[2][1]).get_distincts_labels()
    _plot_confusion_matrix(_pipeline.confusion_matrix, _input, _list_labels)

2023-04-02 11:13:16.583695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 11:13:19.421075: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13119 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS ex

In [ ]:
df_global_report